In [2]:
import openai
import json
import os
import pandas as pd

In [420]:
TEMPLATE_COT= '''

    Given an example input and output dataset, learn how the transformation performed from the provided input dataset to the output dataset. 

    input dataset: {input_list} 

    output dataset: {output_list}

    Pay attention to the size difference between the input and output dataset.

    You should think step by step how the transformation was performed.

    you should generate python code to reproduce the transformation.
    
    In the python code only print out the transformed dataset, no other information.

    you are also given a test set {test_list} where you should include the test set in your python code

    '''

TEMPLATE_C= '''

    Given an example input and output dataset, learn how the transformation performed from the provided input dataset to the output dataset. 
        
    Pay attention to the size difference between the input and output dataset.
    
    Generate python function with no explanations needed. 
        
    input dataset: {input_list} 
    output dataset: {output_list}
    
    you are also given a test set {test_list} where you should include the test set in your python code as the function input
    Do not include input database in your code output
    
    '''
TEMPLATE_6= '''

    Given an example input and output dataset, learn how the transformation performed from the provided input dataset to the output dataset. 
        
    Pay attention to the size difference between the input and output dataset.
    
    Generate python function with no explanations needed. 
        
    input dataset: {input_list} 
    output dataset: {output_list}
    
    you are also given a test set {test_list} where you should include the test set in your python code as the function input
    Do not include input database in your code output

    Note: The element from first row of input dataset will be the first element in each row of the output, follows by product
    
    '''
TEMPLATE_26= '''

    Given an example input and output dataset, learn how the transformation performed from the provided input dataset to the output dataset. 
        
    Pay attention to the size difference between the input and output dataset.
    
    Generate python function with no explanations needed. 
        
    input dataset: {input_list} 
    output dataset: {output_list}
    
    you are also given a test set {test_list} where you should include the test set in your python code as the function input
    Do not include input database in your code output

    Note: change from 1 element a sublist to 4 element per sublist
    
    '''
TEMPLATE_29= '''

    Given an example input and output dataset, learn how the transformation performed from the provided input dataset to the output dataset. 
        
    Pay attention to the size difference between the input and output dataset.
    
    Generate python function with no explanations needed. 
        
    input dataset: {input_list} 
    output dataset: {output_list}
    
    you are also given a test set {test_list} where you should include the test set in your python code as the function input
    Do not include input database in your code output

    Note: combine 2 two from the input to 1 row for the output, but only keep the first word in the first row and all the numerical values
    
    '''
TEMPLATE_LANGUAGE = '''

    Given an example input and output dataset, learn how the transformation performed from the provided input dataset to the output dataset. 
        
    Pay attention to the size difference between the input and output dataset.
    
    Generate python function with no explanations needed. 
        
    input dataset: {input_list} 
    output dataset: {output_list}
    
    you are also given a test set {test_list} where you should include the test set in your python code as the function input
    Do not include input database in your code output

    Note: You should use information you have with this problem such as countries respective ISO language code eg ["Arabic = ar"].
    
    '''
TEMPLATE_KG= '''

    Given an example input and output dataset, learn how the transformation performed from the provided input dataset to the output dataset. 
    Common operations in data cleaning and data wrangling include: identifying and removing duplicate data, handling missing values, 
    filtering outliers, standardizing data formats, data type conversion, data aggregation, joining datasets, data validation, 
    and data enrichment by adding information from external sources. Note you should not use the datetime package.
        
    Pay attention to the size difference between the input and output dataset.
    
    Generate python function with no explanations needed. 
        
    input dataset: {input_list} 
    output dataset: {output_list}
    
    you are also given a test set {test_list} where you should include the test set in your python code as the function input
    Do not include input database in your code output
    '''
TEMPLATE_KG2= '''
    Given an example input and output dataset, learn the transformation applied to convert the provided input dataset into the output dataset.

    Data transformations can include various cleaning and wrangling operations, such as identifying and removing duplicate data, 
    handling missing values, filtering outliers, standardizing data formats, performing data type conversion, data aggregation, 
    joining datasets, validating data integrity, or enriching the dataset by adding information from external sources. 
    However, avoid using any datetime-specific packages for this task.

    When analyzing the transformation, take note of potential changes in the dataset's structure, size, or format between input and output.

    Based on these transformations, generate a Python function that will take a new test set as input and apply the same transformation. 
    Include test_list as the function parameter and ensure the code only outputs the function definition. Do not include the original input_list or output_list directly in the code.
    
    input dataset: {input_list}
    output dataset: {output_list}
    test set: {test_list}
    '''
TEMPLATE_KG_LONG = ''' 
    Given an example input and output dataset, learn the transformations applied to convert the input dataset into the output dataset. These transformations may involve various data cleaning and wrangling tasks that standardize and prepare the data for analysis. Here are some common transformations to consider, along with examples where appropriate:

    Removing Duplicate Data: Detect and remove repeated entries based on specific columns or across the entire dataset.
    Example: If the input dataset contains multiple rows for the same ID, the output dataset may include only the first instance.

    Handling Missing Values: Fill or remove missing data based on the output dataset. You may need to impute values with the mean, median, or mode, or drop rows/columns entirely if missing data is excessive.
    Example: If the input dataset has rows with missing ages, the output dataset might replace them with the average age.

    Filtering Outliers: Remove or modify extreme values that are beyond an acceptable range.
    Example: If a numerical column has values that are three standard deviations away from the mean, they might be removed or capped at a specific threshold.

    Standardizing Data Formats: Ensure consistency in text formats, date formats, and number formats.
    Example: Converting all names to title case (e.g., "john doe" to "John Doe") or ensuring all dates follow the "YYYY-MM-DD" format.

    Data Type Conversion: Change column data types as required by the output dataset, such as converting strings to numbers or dates to strings.
    Example: If a column contains numeric values stored as strings in the input dataset, the output dataset might have them converted to integers or floats.

    Data Aggregation: Combine data at a higher level, like summing, averaging, or grouping by a specific attribute.
    Example: If the input dataset contains daily sales, the output dataset might aggregate these values to show monthly or weekly totals.

    Joining Datasets: Merge data from additional sources if the output dataset contains extra columns not present in the input dataset.
    Example: Joining a product details dataset with the input dataset based on a product ID to include product names in the output.

    Take note of changes in size, format, or structure between the input and output datasets. Also, ensure that any transformations are consistent with observed changes across multiple examples if provided.

    Based on these transformations, generate a Python function that will take a new test set as input and apply the same transformation. 
    Include test_list as the function parameter and ensure the code only outputs the function definition. Do not include the original input_list or output_list directly in the code.
    
    input dataset: {input_list}
    output dataset: {output_list}
    test set: {test_list}
    '''
TEMPLATE_WITH_FUNC = '''
    Given an example input and output dataset, learn the transformations applied to convert the input dataset into the output dataset. 
    These transformations may involve various data cleaning and wrangling tasks that standardize and prepare the data for analysis. 
    Here are some common transformations to consider, along with examples where appropriate, some sample reference functions are provided but you are not limited or have to use the provided functions:

    # Combine the first two rows into one by concatenation
    def combine_first_two_rows(data):
        """
        Combines the first two rows in a 2D list by concatenating them.
        Returns a new 2D list with the combined row.
        """
        if len(data) < 2:
            raise ValueError("Not enough rows to combine.")
        new_row = data[0] + data[1]
        new_data = [new_row] + data[2:]
        return new_data

    # Flip the first row and the leftmost column
    def flip_first_row_and_leftmost_col(data):
        """
        Flips the first row and the leftmost column in a 2D list.
        Returns a new 2D list with the flipped values.
        """
        if not data or not data[0]:
            return data
        num_rows = len(data)
        num_cols = len(data[0])

        # Extract first row and leftmost column
        first_row = data[0]
        leftmost_col = [data[i][0] for i in range(num_rows)]

        # Swap the values
        new_data = [[data[i][j] for j in range(num_cols)] for i in range(num_rows)]
        new_data[0] = leftmost_col
        for i in range(num_rows):
            new_data[i][0] = first_row[i] if i < len(first_row) else None  # Handle any length mismatch

        return new_data

    # Transpose the 2D list
    def transpose(data):
        """
        Transposes a 2D list, flipping rows and columns.
        """
        return [list(row) for row in zip(*data)]

    # Rotate the 2D list 90 degrees clockwise
    def rotate_90_clockwise(data):
        """
        Rotates the 2D list 90 degrees clockwise.
        """
        return [list(row) for row in zip(*data[::-1])]

    # Swap the first two columns
    def swap_first_two_columns(data):
        """
        Swaps the first two columns in a 2D list.
        """
        if len(data[0]) < 2:
            raise ValueError("Not enough columns to swap.")
        return [[row[1], row[0]] + row[2:] for row in data]

    # Add the first two rows element-wise
    def add_first_two_rows(data):
        """
        Adds the first two rows element-wise in a 2D list.
        Returns a new row and a modified 2D list.
        """
        if len(data) < 2:
            raise ValueError("Not enough rows to add.")
        new_row = [data[0][i] + data[1][i] for i in range(len(data[0]))]
        new_data = [new_row] + data[2:]
        return new_data

    # Remove duplicate elements in each row of the 2D list
    def remove_duplicates(data):
        """
        Removes duplicate elements in each row of a 2D list.
        Returns a new 2D list with duplicates removed, preserving the original order.
        """
        return [list(dict.fromkeys(row)) for row in data]


    Based on these transformations, generate a Python function that will take a new test set as input and apply the same transformation. 
    Include test_list as the function parameter and ensure the code only outputs the function definition. Do not include the original input_list or output_list directly in the code.
    
    input dataset: {input_list}
    output dataset: {output_list}
    test set: {test_list}
    '''
### Add operations to the prompt to make it more specific
### run foofah base on fixed prompt
### LLM generate code -> ask LLM to identify the error -> regenerate the code with the error fixed ###

In [3]:
client = AzureOpenAI(
   api_key='17b5615f064c4a899a308979d2f195e7',
   api_version="2024-02-15-preview",
   azure_endpoint="https://wed-aiq-aoai-swc.openai.azure.com/",
)

In [4]:
client = openai.Client(
    api_key='sk-v2jfjIB3eI39LpLh4d060e314f534d4f85C74169E7164c76',
    base_url = "https://open.xiaojingai.com/v1")

In [3]:
def read_in_data(file_name):
    test_data = None
    input_data = [''] * 2
    output_data = [''] * 2
    with open(file_name, 'rb') as f:
        test_data = json.load(f)
            
    input_data[0] = test_data['InputTable']
    input_data[1] = test_data['OutputTable']
    output_data[0] = test_data['TestingTable']
    output_data[1] = test_data['TestAnswer']


    return input_data, output_data

In [6]:
def get_ouput(content):
    settings = {
            "model": "gpt-4o",
            "temperature": 0,
            "seed": 1,
        }
    answer = []
    message = [
        {"role": "user", "content": content},
    ]
    response = client.chat.completions.create(
        messages=message, stream=False, **settings
    )
    if response.choices:
        client_response = response.choices[0].message.content
        answer.append(client_response)
    return client_response

In [7]:
output_path = 'foofah_kg.json'

In [ ]:
from collections import defaultdict
def gpt_output():
        result= []
        #rerun = [47,37,29,26,6,'crime_data_wrangler','potters_wheel_unfold','proactive_wrangling_fold']
        l = [2, 7, 17, 19, 22,26,29, 30,33,34,37,40,43,44,46,47,'potters_wheel_unfold','proactive_wrangling_complex']
        time = [6,12,51]
        name_list = ["craigslist_data_wrangler", "crime_data_wrangler", "potters_wheel_divide", "potters_wheel_fold" ,
                     "potters_wheel_fold_2", "potters_wheel_merge_split", "potters_wheel_split_fold", "potters_wheel_unfold", 
                     "potters_wheel_unfold2", "proactive_wrangling_fold", "proactive_wrangling_complex", "reshape_table_structure_data_wrangler"]
        for j in l:
            for p in range(1):
                for i in range(1,6):
                    path = '../../data/foofah/foofah/exp0_'+str(j) + '_'+ str(i)+ '.txt'
                    #path = '../../data/foofah/Transformation.Text/Language.00000'+str(i)+'.txt'
                    input_data, test_data = read_in_data(path)
                    print('on file', j,i)
                    prompt = TEMPLATE_WITH_FUNC.format(
                        input_list = input_data[0],
                        output_list = input_data[1],
                        test_list = test_data[0]
                    )
                    answer = get_ouput(prompt)
                    
                    # code = answer.split('```python')[1]
                    # code = code.split('```')[0]
                        
                    result.append({'foofah_kg'+str(j)+'_'+ str(1): answer})
                    with open(output_path, "a") as f:
                        json.dump({'foofah_kg'+str(j)+'_'+ str(1): answer}, f)

        # path = '../data/foofah/Transformation.Text/Address.000002.json'
        # input_data, test_data = read_in_data(path)
        # llm = self.environemnt_setup()

        # p_tutorial= PromptTemplate(input_variables=['input_list', 'output_list'],
        #                             template=template_c)
        # chain1 = LLMChain(llm = llm, prompt = p_tutorial)
        # with get_openai_callback() as cb:
        #     tutorial = chain1.run({'input_list': input_data[0], 'output_list': input_data[1]})
        #     print(cb.total_tokens)
                
        # result.append(tutorial)
                
        return result

In [ ]:
gpt_output()

In [14]:
with open(output_path, "r") as f:
    data = json.load(f)


In [2]:
import io
import contextlib

In [3]:
def run_code(code):
    output = io.StringIO()

    # Use contextlib.redirect_stdout to redirect print statements to the StringIO object
    try:
        with contextlib.redirect_stdout(output):
            exec(code)

        # Get the output as a string
        captured_output = output.getvalue()
    except:
        captured_output = []
    return captured_output

In [8]:
import json
code = "\ndef transform_data(input_list):\n    \"\"\"\n    Transforms the input dataset into the required output dataset by concatenating elements from consecutive rows.\n    \"\"\"\n    # Initialize an empty list to store the transformed data\n    output_list = []\n\n    # Process every pair of rows in the input list\n    for i in range(0, len(input_list), 2):\n        # Concatenate relevant parts of the two consecutive rows\n        combined_row = input_list[i][:3] + input_list[i+1][1:5]  # Adjusted to [1:5] instead of [1:6]\n        # Append the combined row to the output list\n        output_list.append(combined_row)\n    \n    return output_list\n\n# Define the input list\ninput_list = [['3099', '905', ' AUST 4WD CUST ACT', '', '', '', ''], ['NO.14', 'NO.14', 'Full Copies', '6.7839', '2', '* *', '0'], ['3200', '906', 'AUST HOUSE & GARDEN', '', '', '', ''], ['9-Jun', '9-Jun', 'Covers Only', '4.7385', '1', '* *', '0'], ['3167', '906', 'AUST PERSONAL COMPUTER', '', '', '', ''], ['9-Jun', '9-Jun', 'Covers Only', '6.7839', '3', '* *', '0'], ['2929', '924', 'AUTO ACTION', '', '', '', ''], ['#1344', '#1344', 'Covers Only', '3.8181', '3', '* *', '0'], ['5356', '901', 'BEAUT BEADED BAGS', '', '', '', ''], ['# 1', '# 1', 'Full Copies', '10.1929', '3', '* *', '0']]\n\n# Transform the input list\noutput_list = transform_data(input_list)\n\n# Print the transformed output list\nprint(output_list)\n"

In [ ]:
x = run_code(code)
i, t = read_in_data('../../data/foofah/foofah/exp0_29_3.txt')


NameError: name 'run_code' is not defined

In [10]:
i[1]

[['3099', '905', ' AUST 4WD CUST ACT', 'NO.14', 'Full Copies', '6.7839', '2'],
 ['3200', '906', 'AUST HOUSE & GARDEN', '9-Jun', 'Covers Only', '4.7385', '1'],
 ['3167',
  '906',
  'AUST PERSONAL COMPUTER',
  '9-Jun',
  'Covers Only',
  '6.7839',
  '3'],
 ['2929', '924', 'AUTO ACTION', '#1344', 'Covers Only', '3.8181', '3'],
 ['5356', '901', 'BEAUT BEADED BAGS', '# 1', 'Full Copies', '10.1929', '3']]

In [14]:
pre = eval(x)
print(pre == i[1])

True


In [ ]:
test = []
j_idx = 0
i_idx = 1
l = [2, 7, 17, 19, 22,26,29, 30,33,34,37,40,43,44,46,47,'potters_wheel_unfold','proactive_wrangling_complex']
for d in data:
    for key, value in d.items():
        print(key)
        dataset = key[:-2]
        # path = '../../data/foofah/foofah/date_exp0_'+dataset+ '.txt'
        # input_data, test_data = read_in_data(path)
        # input_data = test_data[0]
        # input_dataset = test_data[0]
        if i_idx == 6:
            i_idx = 1
            j_idx += 1
        k = str(l[j_idx])+ '_'+ str(i_idx)
        code = value.split('```python')[1]
        code = code.split('```')[0]
        value = run_code(code)
        test.append({k: value})
        i_idx += 1
        


In [6]:
input_data, test_data = read_in_data('../../data/foofah/foofah/exp0_29_3.txt')
df = pd.DataFrame(input_data[1])
df

,0,1,2,3,4,5,6,7,8
0,ABC,"766,469",74,"703,255",70,"631,646",59,"2,101,370",203
1,DEF,"776,996",60,"1,532,159",76,"494,919",42,"2,804,074",178
2,XYZ,"832,414",67,"897,949",63,"712,365",52,"2,442,728",182


In [8]:
df = pd.DataFrame(test_data[1])
df

,0,1,2,3,4,5,6,7,8
0,DEF,"776,996",60,"1,532,159",76,"494,919",42,"2,804,074",178
1,FYI,"818,331",79,"349,399",12,"552,587",40,"1,720,317",131
2,AFK,"454,724",71,"568,168",20,"661,672",62,"1,684,564",153


In [421]:
def gpt_output():
        result= []
        #rerun = [47,37,29,26,6,'crime_data_wrangler','potters_wheel_unfold','proactive_wrangling_fold']
        l = [2]
        time = [6,12,51]
        name_list = ["craigslist_data_wrangler", "crime_data_wrangler", "potters_wheel_divide", "potters_wheel_fold" ,
                     "potters_wheel_fold_2", "potters_wheel_merge_split", "potters_wheel_split_fold", "potters_wheel_unfold", 
                     "potters_wheel_unfold2", "proactive_wrangling_fold", "proactive_wrangling_complex", "reshape_table_structure_data_wrangler"]
        for j in l:
            for p in range(1):
                for i in range(1,6):
                    path = '../../data/foofah/foofah/exp0_'+str(j) + '_'+ str(i)+ '.txt'
                    #path = '../../data/foofah/Transformation.Text/Language.00000'+str(i)+'.txt'
                    input_data, test_data = read_in_data(path)
                    print('on file', j,i)
                    prompt = TEMPLATE_WITH_FUNC.format(
                        input_list = input_data[0],
                        output_list = input_data[1],
                        test_list = test_data[0]
                    )
                    print("PROMPT:", prompt)
                    # answer = get_ouput(prompt)
                    # print("ANSWER:", answer)
gpt_output()

on file 2 1
PROMPT: 
    Given an example input and output dataset, learn the transformations applied to convert the input dataset into the output dataset. 
    These transformations may involve various data cleaning and wrangling tasks that standardize and prepare the data for analysis. 
    Here are some common transformations to consider, along with examples where appropriate, some sample reference functions are provided but you are not limited or have to use the provided functions:

    # Combine the first two rows into one by concatenation
    def combine_first_two_rows(data):
        """
        Combines the first two rows in a 2D list by concatenating them.
        Returns a new 2D list with the combined row.
        """
        if len(data) < 2:
            raise ValueError("Not enough rows to combine.")
        new_row = data[0] + data[1]
        new_data = [new_row] + data[2:]
        return new_data

    # Flip the first row and the leftmost column
    def flip_first_row_and

In [415]:
# Revised function to handle out-of-range errors and ensure conditions are correctly evaluated
def transform_dataset(input_data):
    result = []
    i = 0
    while i < len(input_data) - 1:  # Ensure we don't go out of bounds
        # Check if the next row meets the criteria for combining with the current row
        next_row = input_data[i + 1]
        if (next_row[0].startswith('#') or next_row[0].startswith('NO') or next_row[0][0].isdigit() or next_row[0].count('-') == 1):
            transformed_row = [
                input_data[i][0],  # Column 1
                input_data[i][1],  # Column 2
                input_data[i][2],  # Column 3
                next_row[0],       # Column 4 from next row
                next_row[2],       # Column 5 from next row
                next_row[3],       # Column 6 from next row
                next_row[4]        # Column 7 from next row
            ]
            result.append(transformed_row)
            i += 2  # Skip the next row as it's already used
        else:
            i += 1  # Move to the next row if no transformation is applied
    return result

# Run the revised function on the test data

# Run the function on the test data
transformed_test_data = transform_dataset(test_data[0])
print(transformed_test_data == test_data[1])


False


In [416]:
transformed_test_data

[['5950', '901', 'BETTER PHOTOGRAPHY', '#55', 'Full Copies', '8.8293', '3'],
 ['60009', '905', 'BLISS', '9-Apr', 'Covers Only', '4.0567', '1'],
 ['6454', '911', 'CARRY ON DVD NAT', 'NO 32', 'Full Copies', '13.6019', '1'],
 ['7127', '911', 'CL WAR MOVIES NAT', 'NO 30', 'Full Copies', '13.6019', '1'],
 ['WINT09', 'WINT09', 'Full Copies', '5714', 'CRDMKNG STMPNG&PPRCRFT', '', ''],
 ['V14#11', 'V14#11', 'Full Copies', '7663', 'DORA DOLL NAT', '', ''],
 ['NO 24', 'NO 24', 'Full Copies', '1876', 'ECONOMIST THE', '', ''],
 ['13-Jun', '13-Jun', 'Covers Only', '1942', 'ENG WOMANS WEEKLY', '', ''],
 ['16-Jun', '16-Jun', 'Covers Only', '5919', 'FAST FOURS', '', '']]

In [417]:
test_data[1]

[['5950', '901', 'BETTER PHOTOGRAPHY', '#55', 'Full Copies', '8.8293', '3'],
 ['60009', '905', 'BLISS', '9-Apr', 'Covers Only', '4.0567', '1'],
 ['6454', '911', 'CARRY ON DVD NAT', 'NO 32', 'Full Copies', '13.6019', '1'],
 ['7127', '911', 'CL WAR MOVIES NAT', 'NO 30', 'Full Copies', '13.6019', '1'],
 ['5210',
  '902',
  'COSMOPOLITAN PREGNANCY',
  'WINT09',
  'Full Copies',
  '9.2043',
  '3'],
 ['5714',
  '905',
  'CRDMKNG STMPNG&PPRCRFT',
  'V14#11',
  'Full Copies',
  '6.7839',
  '3'],
 ['7663', '911', 'DORA DOLL NAT', 'NO 24', 'Full Copies', '5.4203', '1'],
 ['1876', '924', 'ECONOMIST THE', '13-Jun', 'Covers Only', '7.1589', '4'],
 ['1942', '923', 'ENG WOMANS WEEKLY', '16-Jun', 'Covers Only', '2.2499', '1'],
 ['5919', '906', 'FAST FOURS', '9-Jun', 'Full Copies', '6.7839', '2']]

In [409]:
path = '../../data/foofah/foofah/exp0_2_5.txt'
input_data, test_data = read_in_data(path)
x =[['5356', '901', 'BEAUT BEADED BAGS', '# 1', 'Full Copies', '10.1929', '3'],
 ['5950', '901', 'BETTER PHOTOGRAPHY', '#55', 'Full Copies', '8.8293', '3'],
 ['60009', '905', 'BLISS', '9-Apr', 'Covers Only', '4.0567', '1'],
 ['6454', '911', 'CARRY ON DVD NAT', 'NO 32', 'Full Copies', '13.6019', '1'],
 ['7127', '911', 'CL WAR MOVIES NAT', 'NO 30', 'Full Copies', '13.6019', '1'],
 ['5210', '902', 'COSMOPOLITAN PREGNANCY', 'WINT09', 'Full Copies', '9.2043', '3'],
 ['5714', '905', 'CRDMKNG STMPNG&PPRCRFT', 'V14#11', 'Full Copies', '6.7839', '3'],
 ['7663', '911', 'DORA DOLL NAT', 'NO 24', 'Full Copies', '5.4203', '1'],
 ['1876', '924', 'ECONOMIST THE', '13-Jun', 'Covers Only', '7.1589', '4'],
 ['1942', '923', 'ENG WOMANS WEEKLY', '16-Jun', 'Covers Only', '2.2499', '1'],
 ['5919', '906', 'FAST FOURS', '9-Jun', 'Full Copies', '6.7839', '2']]


print(transformed_test_data == test_data[1])
input_data[1][0]

False


['3099', '905', ' AUST 4WD CUST ACT', 'NO.14', 'Full Copies', '6.7839', '2']

In [55]:
def transform_data(test_set):
    # Extract subjects
    subjects = list(set(entry[1] for entry in test_set))
    subjects.sort()

    # Extract names
    names = list(set(entry[0] for entry in test_set))
    names.sort()

    # Initialize the output with headers
    output = [[''] + subjects]

    # Initialize dictionary for easy lookup
    scores_dict = {(name, subject): '' for name in names for subject in subjects}
    
    # Fill the dictionary with actual scores
    for entry in test_set:
        name, subject, score = entry
        scores_dict[(name, subject)] = score

    # Populate the output dataset using the dictionary
    for name in names:
        output.append([name] + [scores_dict[(name, subject)] for subject in subjects])

    return output

# Example test execution
test_set = [['Anna', 'Math', '43'], ['Anna', 'French', '78'], ['Bob', 'English', '96'], ['Bob', 'French', '54'], ['Tom', 'Math', '90'], ['Tom', 'French', '85'], ['Rob', 'English', '87'], ['Rob', 'French', '92']]
result = transform_data(test_set)
for row in result:
    print(row)

['', 'English', 'French', 'Math']
['Anna', '', '78', '43']
['Bob', '96', '54', '']
['Rob', '87', '92', '']
['Tom', '', '85', '90']


In [64]:
df = pd.DataFrame(test_data[1])
df

,0,1,2,3
0,,Math,French,English
1,Anna,43,78,
2,Bob,,54,96
3,Tom,90,85,
4,Rob,,92,87


### Notes:
 1. in the prompt, give a list of example operations that it wants to prioritze 

 (could also let gpt to generate the informations about the background about PBE)

In [24]:
with open('foofah_kg_result.json', "w") as f:
    json.dump(test, f)

In [17]:
import pandas as pd 
path = '../../data/foofah/foofah/exp0_3_2.txt'
input_data, test_data = read_in_data(path)
df_1 = pd.DataFrame(input_data[1])
df_1

,0,1,2,3
0,33195,Promotion,2/13/2009,821017
1,33195,Promotion,3/30/2009,65320
2,50999,Inc Actvty - Year end,2/16/2009,64023
3,50999,Inc Actvty - Year end,3/2/2009,64519
4,50999,Inc Actvty - Year end,3/2/2009,64519
5,50999,Inc Actvty - Year end,3/16/2009,164351


In [388]:
import pandas as pd
df = pd.DataFrame(input_data[1])
df

,0,1,2,3,4,5,6
0,3099,905,AUST 4WD CUST ACT,NO.14,Full Copies,6.7839,2
1,3200,906,AUST HOUSE & GARDEN,9-Jun,Covers Only,4.7385,1
2,3167,906,AUST PERSONAL COMPUTER,9-Jun,Covers Only,6.7839,3


In [389]:
df = pd.DataFrame(test_data[0])
df

,0,1,2,3,4,5,6
0,2929,924,AUTO ACTION,,,,
1,#1344,#1344,Covers Only,3.8181,3,* *,0
2,5356,901,BEAUT BEADED BAGS,,,,
3,# 1,# 1,Full Copies,10.1929,3,* *,0
4,5950,901,BETTER PHOTOGRAPHY,,,,
5,#55,#55,Full Copies,8.8293,3,* *,0
6,60009,905,BLISS,,,,
7,9-Apr,9-Apr,Covers Only,4.0567,1,* *,0
8,6454,911,CARRY ON DVD NAT,,,,
9,NO 32,NO 32,Full Copies,13.6019,1,* *,0


In [390]:
df = pd.DataFrame(test_data[1])
df

,0,1,2,3,4,5,6
0,2929,924,AUTO ACTION,#1344,Covers Only,3.8181,3
1,5356,901,BEAUT BEADED BAGS,# 1,Full Copies,10.1929,3
2,5950,901,BETTER PHOTOGRAPHY,#55,Full Copies,8.8293,3
3,60009,905,BLISS,9-Apr,Covers Only,4.0567,1
4,6454,911,CARRY ON DVD NAT,NO 32,Full Copies,13.6019,1
5,7127,911,CL WAR MOVIES NAT,NO 30,Full Copies,13.6019,1
6,5210,902,COSMOPOLITAN PREGNANCY,WINT09,Full Copies,9.2043,3
7,5714,905,CRDMKNG STMPNG&PPRCRFT,V14#11,Full Copies,6.7839,3
8,7663,911,DORA DOLL NAT,NO 24,Full Copies,5.4203,1
9,1876,924,ECONOMIST THE,13-Jun,Covers Only,7.1589,4


In [348]:
tmp = []
ans = []
i = 0
for d in test_data[0]:
    p = d[:6]
    for j in range(6, len(d), 4):
        tmp  = p.copy()
        tmp.append(d[j])
        tmp.append(d[j+1])
        tmp.append(d[j+2])
        tmp.append(d[j+3])
        ans.append(tmp)
print("'TestAnswer':", ans)

'TestAnswer': []


In [380]:
def transform_data_corrected(data):
    # Step 1: Separate main records and details records
    main_records = [data[i] for i in range(0, len(data), 2)]
    details_records = [data[i] for i in range(1, len(data), 2)]
    
    # Step 2: Correctly combine main records and details records by position with specified column selection
    transformed_data = [
        main[:3] + [detail[0], detail[2], detail[3], detail[4]]
        for main, detail in zip(main_records, details_records)
    ]
    
    return transformed_data
x = transform_data_corrected(test_data[0])
print(x == test_data[1])

True


In [117]:
output

[['Anthony, Scott A.', ['Partner'], ['Palo Alto'], ['Mergers & Acquisitions']],
 ['Baudler, Mark B.', ['Partner'], ['Palo Alto'], ['Mergers & Acquisitions']],
 ['Berger, David J.',
  ['Partner'],
  ['San Francisco, Palo Alto'],
  ['Mergers & Acquisitions']]]

In [118]:
test_data[1]

[['Anthony, Scott A.', 'Partner', 'Palo Alto', 'Mergers & Acquisitions'],
 ['Baudler, Mark B.', 'Partner', 'Palo Alto', 'Mergers & Acquisitions'],
 ['Berger, David J.',
  'Partner',
  'San Francisco, Palo Alto',
  'Mergers & Acquisitions']]

In [111]:
input_table = test_data[0]
output_table = []

# Process each row in the input_table, skipping the header row
for row in input_table[1:]:
    year = row[0]
    catnum = row[1]
    comments = row[2]
    
    # Loop through months (1 to 12) and create separate rows for each month
    for i in range(3, 15):
        month = input_table[0][i]  # Get the month number from the header row
        value = row[i]  # Get the value corresponding to the month
        output_table.append([year, catnum, comments, month, value])

# Display the OutputTable
output_table
# output
if output_table == test_data[1]:
    print('correct')
else:
    print(len(output_table), len(test_data[1]))

correct


In [103]:
test_data[1]

[['', 'Anna', 'Davis'],
 ['', 'Joan', 'Marsh'],
 ['', 'Daenerys', 'Rhaenys'],
 ['Stark, Eddard', '', ''],
 ['', 'Robb', 'Sansa'],
 ['Lannister, Tywin', '', ''],
 ['', 'Jaime', 'Cersei'],
 ['Hoster, Tully', '', ''],
 ['', 'Lysa', 'Catelyn']]

In [58]:
test_data[1]

[['14:54', '-72'],
 ['14:54', '-71'],
 ['14:54', '-90'],
 ['14:54', '-89'],
 ['14:54', '-43'],
 ['14:54', '-76'],
 ['14:55', '-45'],
 ['14:55', '-87'],
 ['14:55', '-98'],
 ['14:55', '-90'],
 ['14:55', '-78'],
 ['14:55', '-76'],
 ['14:55', '-87'],
 ['14:55', '-87'],
 ['14:55', '-98'],
 ['14:55', '-45'],
 ['14:55', '-97'],
 ['14:55', '-45'],
 ['14:66', '-76'],
 ['14:66', '-98'],
 ['14:66', '-45']]

### eval

In [25]:
with open('foofah_kg_result.json', "r") as f:
    result = json.load(f)


In [26]:
dest_folder = './error/'
source_path = '../output/'
from collections import defaultdict
def calculate_acc():
    acc = defaultdict(list)
    for d in result:
        for key, value in d.items():
            print(key)
            dataset = key[:-2]
            try:
                pre = eval(value)
            except:
                acc[dataset].append([key, 0])
                continue
            path = '../../data/foofah/foofah/exp0_'+key+ '.txt'
            input_data, test_data = read_in_data(path)
            # input_path = source_path+model+'foofah/output_'+str(j) + '_'+ str(i)+ '.json'
            #     llama70 = read_llm_output_data(input_path)
            #     # gpt_3_5 = read_llm_output_data('../output/Llama-2-13b-chat-hf/foofah/output_data_0_'+str(j) + '_'+ str(i)+ '.json')
            #     gpt_4_0 = read_llm_output_data('../output/chat_gpt_4.0/foofah/new_output_data_0_'+str(j) + '_'+ str(i)+ '.json')
            #     # if os.path.isfile('../output/foofah/foofah/exp0_results_'+str(j) + '_'+ str(i)+ '.txt'):
            #     #     foofah = read_output_data('../output/foofah/foofah/exp0_results_'+str(j) + '_'+ str(i)+ '.txt')
            #     # else:
            #     #     foofah = [[]]

                #compare output_data and test_data
            print(pre, test_data[1])
            acc_temp = 0
                        # print(j,i)
            ans = test_data[1]

            for k in range(min(len(pre),len(ans))):
                for m in range(min(len(pre[k]),len(ans[k]))):
                    if pre[k][m] == ans[k][m]:
                        acc_temp += 1
            total_values = sum(len(inner_list) for inner_list in ans)
            test_case_acc = acc_temp/total_values
                
            acc[dataset].append([key, test_case_acc])
 

    return(acc)
                
 

In [27]:
acc_result = calculate_acc()

2_1
[['3200', '906', 'AUST HOUSE & GARDEN', 'Covers Only', '4.7385', '1'], ['3167', '906', 'AUST PERSONAL COMPUTER', 'Covers Only', '6.7839', '3'], ['2929', '924', 'AUTO ACTION', 'Covers Only', '3.8181', '3'], ['5356', '901', 'BEAUT BEADED BAGS', 'Full Copies', '10.1929', '3'], ['5950', '901', 'BETTER PHOTOGRAPHY', 'Full Copies', '8.8293', '3'], ['60009', '905', 'BLISS', 'Covers Only', '4.0567', '1'], ['6454', '911', 'CARRY ON DVD NAT', 'Full Copies', '13.6019', '1'], ['7127', '911', 'CL WAR MOVIES NAT', 'Full Copies', '13.6019', '1'], ['5210', '902', 'COSMOPOLITAN PREGNANCY', 'Full Copies', '9.2043', '3'], ['5714', '905', 'CRDMKNG STMPNG&PPRCRFT', 'Full Copies', '6.7839', '3'], ['7663', '911', 'DORA DOLL NAT', 'Full Copies', '5.4203', '1'], ['1876', '924', 'ECONOMIST THE', 'Covers Only', '7.1589', '4'], ['1942', '923', 'ENG WOMANS WEEKLY', 'Covers Only', '2.2499', '1'], ['5919', '906', 'FAST FOURS', 'Full Copies', '6.7839', '2']] [['3200', '906', 'AUST HOUSE & GARDEN', '9-Jun', 'Cover

In [28]:
acc_result

defaultdict(list,
            {'2': [['2_1', 0.42857142857142855],
              ['2_2', 0],
              ['2_3', 0],
              ['2_4', 0.5714285714285714],
              ['2_5', 0.5714285714285714]],
             '7': [['7_1', 0],
              ['7_2', 1.0],
              ['7_3', 0],
              ['7_4', 1.0],
              ['7_5', 1.0]],
             '17': [['17_1', 1.0],
              ['17_2', 0],
              ['17_3', 1.0],
              ['17_4', 1.0],
              ['17_5', 1.0]],
             '19': [['19_1', 1.0],
              ['19_2', 1.0],
              ['19_3', 0],
              ['19_4', 1.0],
              ['19_5', 1.0]],
             '22': [['22_1', 1.0],
              ['22_2', 1.0],
              ['22_3', 1.0],
              ['22_4', 0],
              ['22_5', 1.0]],
             '26': [['26_1', 1.0],
              ['26_2', 1.0],
              ['26_3', 0.0],
              ['26_4', 1.0],
              ['26_5', 1.0]],
             '29': [['29_1', 0.1111111111111111],


In [145]:
with open('gpt_date_acc_result.json', "w") as f:
    json.dump(acc_result, f)

In [29]:
overall_acc = []
for key in acc_result:
    acc = 0
    for i in acc_result[key]:
        acc += i[1]
    overall_acc.append([key, acc/len(acc_result[key])])

In [30]:
list_acc = defaultdict(list)
for i in overall_acc:
    exp = i[0].split('_')[0]
    list_acc[exp].append(i[1])
list_acc

defaultdict(list,
            {'2': [0.3142857142857143],
             '7': [0.6],
             '17': [0.8],
             '19': [0.8],
             '22': [0.8],
             '26': [0.8],
             '29': [0.28888888888888886],
             '30': [0.6666666666666666],
             '33': [0.45],
             '34': [0.49777777777777776],
             '37': [0.44400000000000006],
             '40': [0.6900000000000001],
             '43': [0.6],
             '44': [1.0],
             '46': [0.9199999999999999],
             '47': [0.8400000000000001],
             'potters': [0.5833333333333334],
             'proactive': [0.8400000000000001]})

In [31]:
for e in list_acc:
    print(e, sum(list_acc[e])/len(list_acc[e]))

2 0.3142857142857143
7 0.6
17 0.8
19 0.8
22 0.8
26 0.8
29 0.28888888888888886
30 0.6666666666666666
33 0.45
34 0.49777777777777776
37 0.44400000000000006
40 0.6900000000000001
43 0.6
44 1.0
46 0.9199999999999999
47 0.8400000000000001
potters 0.5833333333333334
proactive 0.8400000000000001


In [1]:
import pandas as pd